# Portfolio management

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import lib as l

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#268040', '#800B00', '#A13808', '#464CC2'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2'])

#### Reading data

In [3]:
dquote = pd.read_csv('br_quotes.csv')
dstocks = pd.read_csv('br_stocks.csv')
dnu = pd.read_csv('br_invest.csv')
dnu['Data'] = dnu['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dprices = pd.read_csv('br_prices.csv')
dport = pd.read_csv('br_portfolio.csv')
dport['Date'] = dport['Date'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dprices = pd.read_csv('br_prices.csv')

In [4]:
dflow = dnu.groupby('ID').apply(l.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow = dflow.sort_values(by=['Rendimento anual'], ascending=False).reset_index(drop=True)

#### Calculating stocks return

In [5]:
dreturn = pd.DataFrame()
current_year = dt.date.today().year
dreturn['Ticker'] = dquote['Ticker']
dreturn['Predicted'] = [0 for j in dquote['Ticker']]
weights = [0.05, 0.1, 0.15, 0.3, 0.4]
for i in range(5):
    year_i = current_year-4+i
    dreturn[str(year_i)] = (dquote[str(year_i)] - dquote[str(year_i-1)]) / dquote[str(year_i-1)]
    dreturn['Predicted'] += dreturn[str(year_i)] * weights[i]

# Screener

In [6]:
assets = ['USA', 'EU', 'BR', 'EM', 'Bonds', 'Cash', 'Commod']
industry = ['Basic Mat', 'Cons Cycl', 'Finance', 'Real Estate', 'Telecom', 'Energy', 'Indust', 'Tech', 'Cons Non-Cycl', 'Health', 'Util']

dfull = l.calc_stocks_feat(dreturn, dstocks, dquote, assets, industry)

screen = ['Ticker', 'Score', 'Price', 'Return', 'Risk', 'Tax', 'YTD', 'Cat', 'Geo', 'Domain', 'Asset']
l.change_df_prop(dfull[screen].sort_values('Score', ascending=False).reset_index(drop=True))

Ticker,Score,Price,Return,Risk,Tax,YTD,Cat,Geo,Domain,Asset
BBSD11,7.99369,95.47,29.99%,17.73%,0.5%,34.2%,3,USA,Indust,Equity
DIVO11,7.93795,69.3,30.75%,19.25%,0.5%,40.14%,3,USA,Util,Equity
IVVB11,7.84647,140.21,23.75%,12.18%,0.24%,43.09%,2,USA,Tech,Equity
BRAX11,7.3552,96.05,24.94%,17.38%,0.2%,29.45%,1,BR,Finance,Equity
BOVB11,7.17266,116.26,24.39%,17.73%,0.2%,27.13%,1,BR,Finance,Equity
BOVV11,7.12266,116.26,24.39%,17.73%,0.3%,27.13%,1,BR,Finance,Equity
PIBB11,6.92094,196.39,23.3%,17.68%,0.06%,25.97%,1.1,BR,Finance,Equity
GOVE11,6.76352,50.66,26.51%,22.55%,0.5%,31.07%,1,BR,Finance,Equity
MATB11,5.70007,36.07,19.97%,22.55%,0.5%,12.16%,4,BR,Basic Mat,Equity


# Portfolio Overview

In [7]:
dport_full = l.calc_portfolio(dport, dfull, assets, industry)
dport_bonds = l.add_bonds(dport_full, dflow, assets)
dport_bonds = l.calc_distribution(dport_bonds, assets, industry)

## Current state

In [8]:
dtotal = pd.DataFrame({
    'Value': [(dport_full['Price'] * dport_full['Shares']).sum()],
    'Profit': [dport_full['Profit'].sum()],
    'Yield': [l.int2pct(dport_full['Profit'].sum() / (dport_full['Buy'] * dport_full['Shares']).sum())]
})
dtotal['Year profit'] = dport_full['Year profit'].sum()
dtotal['Annual yield'] = (dtotal['Year profit'] / dtotal['Value']).apply(l.int2pct)
l.change_df_prop(dtotal, 28)

Value,Profit,Yield,Year profit,Annual yield
3764.7,191.4,5.36%,2036.94,54.11%


## Portfolio components

In [9]:
status = ['Ticker', 'Value', 'Profit', 'Yield %']
l.change_df_prop(dport_bonds[status], 24)

Ticker,Value,Profit,Yield %
IVVB11,1402.1,94.1,7.19%
BRAX11,960.5,85.5,9.77%
IVVB11,1402.1,11.8,0.85%
NU,14906.1,122.72,0.4%


In [10]:
revenue = ['Ticker', 'Annual yield %', 'Year profit', 'Tax', 'Annual yield (Liq.) %', 'Year profit (Liq.)']
l.change_df_prop(dport_full[revenue], 24)

Ticker,Annual yield %,Year profit,Tax,Annual yield (Liq.) %,Year profit (Liq.)
IVVB11,51.54%,722.67,18.4%,40.68%,570.413
BRAX11,109.54%,1052.16,17.18%,85.43%,820.524
IVVB11,18.69%,262.109,43.78%,10.13%,142.094


## Actives allocation

In [11]:
dgroup = dport_bonds.groupby('Ticker').sum()
fig = l.plot_pie(dgroup['Value'], domain=dict(x=[0,0.5]), name='Funds')
l.plot_pie(dgroup['Profit'], fig=fig, domain=dict(x=[0.5,1]), name='Profit')

FigureWidget({
    'data': [{'domain': {'x': [0, 0.5]},
              'hole': 0.4,
              'labels': arr…

## Asset allocation

In [12]:
dgroup = dport_bonds.sum()
fig = l.plot_pie(dgroup[assets], domain=dict(x=[0,0.5]), name='Asset')
l.plot_pie(dgroup[industry], fig=fig, domain=dict(x=[0.5,1]), name='Industry')

FigureWidget({
    'data': [{'domain': {'x': [0, 0.5]},
              'hole': 0.4,
              'labels': arr…

# Portfolio Evolution

In [13]:
dport

,Ticker,Buy,Shares,Buy Tax,Date
0,IVVB11,130.800,10,0.400,2019-10-30
1,BRAX11,87.500,10,0.270,2019-11-14
2,IVVB11,139.030,10,0.630,2019-12-12


In [14]:
dport_full

,Ticker,Buy,Shares,Buy Tax,Date,Price,USA,EU,BR,EM,...,Sell Tax,Total Tax,Tax,Yield (Liq.),Annual yield (Liq.),Year profit (Liq.),Yield %,Annual yield %,Yield (Liq.) %,Annual yield (Liq.) %
0,IVVB11,130.800,10,0.400,2019-10-30,140.210,98.880,0.990,0.000,0,...,13.550,17.315,18.4%,0.059,0.407,570.413,7.19%,51.54%,5.87%,40.68%
1,BRAX11,87.500,10,0.270,2019-11-14,96.050,0.000,0.000,99.250,0,...,12.496,14.687,17.18%,0.081,0.854,820.524,9.77%,109.54%,8.09%,85.43%
2,IVVB11,139.030,10,0.630,2019-12-12,140.210,98.880,0.990,0.000,0,...,1.171,5.166,43.78%,0.005,0.101,142.094,0.85%,18.69%,0.48%,10.13%


# Portfolio Simulation